In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch.utils.data import DataLoader

import kornia.geometry.conversions as conversions


from torchinfo import summary
from models import XSwinFusion 
from pose_estimation import PoseDataNPZTorch


WORKDIR = f"{os.getcwd()}/.."
DATA_FOLDER = os.path.join(WORKDIR, "data_folder")
DATASET_NPZ_PATH = os.path.join(DATA_FOLDER, "dataset_npz")


In [2]:
samples = 1_000
resize = (144, 256)
aspect_ratio = True
margin = 12
feature_dims = 64
pretrained = True
B = 10

model = XSwinFusion(feature_dims=feature_dims, resize=resize, pretrained=True)

dataset = None
dataset = PoseDataNPZTorch(DATASET_NPZ_PATH, samples=samples, 
                           resize=resize, aspect_ratio=aspect_ratio, margin=margin)
dataloader = DataLoader(dataset, batch_size=B, shuffle=True)

summary(model, depth=1)


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FCN_ResNet50_Weights.COCO_WITH_VOC_LABELS_V1`. You can also use `weights=FCN_ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Presumed Preloaded NPZ Dataset: /Users/armanommid/Code/CSE/CSE275/HW2/XSwinDiffusion/../data_folder/dataset_npz


Layer (type:depth-idx)                                       Param #
XSwinFusion                                                  --
├─FCN: 1-1                                                   35,355,328
├─PointNet: 1-2                                              622,482
├─Sequential: 1-3                                            12,672
├─Sequential: 1-4                                            265,088
├─Sequential: 1-5                                            3,161
├─Sequential: 1-6                                            2,883
├─Sequential: 1-7                                            2,865
Total params: 36,264,479
Trainable params: 36,260,374
Non-trainable params: 4,105

In [22]:
points = []

for i, (s, t, c, mi, p, sym) in enumerate(dataloader):
    print(sym)


('zinf', 'z4|x4|y4', 'z2', 'zinf', 'z4|x4|y4', 'z4|x4|y4', 'z2|y2|x4', 'z2|y2|x4', 'z2|x2', 'z4')
('z2', 'no', 'z2|x4|y2', 'z2|y2|x4', 'z2|x2', 'z2|x2', 'zinf', 'zinf|x2', 'z2|x2', 'z2|x2')
('z2|x2', 'z2|y2|x4', 'x2', 'zinf|x2', 'zinf', 'z2|x2', 'zinf', 'zinf|x2', 'no', 'z2|x2')
('zinf', 'zinf|x2', 'z2|y2|x4', 'z2|x2', 'x2', 'zinf|x2', 'z2', 'no', 'zinf|x2', 'zinf')
('z2|x2', 'z2', 'z2|y2|x4', 'z2|x2', 'zinf|x2', 'z2|x2', 'z2', 'z2|x2', 'z2', 'no')
('xinf', 'z2|x2', 'z2|x2', 'z2|x4|y2', 'z2|x2', 'zinf|x2', 'z2|x2', 'zinf', 'z2', 'z4|x4|y4')
('z2|x4|y2', 'zinf|x2', 'z4|x4|y4', 'z2', 'zinf', 'z2', 'z2', 'z4|x4|y4', 'zinf|x2', 'z2|x2')
('z4', 'z2|x2', 'z4|x4|y4', 'zinf|x2', 'z2', 'z2|x2', 'zinf|x2', 'z2|y2|x4', 'z2', 'z2')
('z2|y2|x4', 'z2', 'z2', 'zinf|x2', 'z2|x2', 'z2', 'zinf|x2', 'z2|x2', 'zinf|x2', 'z2|x2')
('zinf|x2', 'z2|y2|x4', 'zinf|x2', 'zinf|x2', 'zinf', 'z2|x2', 'z2|y2|x4', 'zinf', 'z2', 'z2|x2')
('zinf|x2', 'z2|x2', 'z4|x4|y4', 'z2', 'zinf|x2', 'z2|x2', 'zinf|x2', 'z2', 'zinf

KeyboardInterrupt: 

In [23]:


def enumerate_symmetries(R, info, inf=24):
    basis = {"x" : torch.tensor([1, 0, 0]), "y" : torch.tensor([0, 1, 0]), "z" : torch.tensor([0, 0, 1])}
    matrices = []
    if info == "no":
        return matrices
    print(info)
    print(info.split("|"))
    for sym_n in info[0].split("|"):
        b = sym_n[0]
        n = int(sym_n[1]) if len(sym[1:]) == 1 else inf
        angles = torch.linspace(0, 2*torch.pi, steps=n)
        for angle in angle:
            matrices.append(
                conversions.angle_axis_to_rotation_matrix(angle * basis[b])
            )
    return matrices



enumerate_symmetries(p[0], sym[0])


z2|x2
['z2', 'x2']


TypeError: linspace() got an unexpected keyword argument 'endpoint'